In [1]:
import pandas as pd
import numpy as np
import os, pdb, sys, pickle
from gudhi import RipsComplex, SimplexTree
import matplotlib.pyplot as plt

# Experimental Design

**Simulate Data Under Heterogeneous Scenarios**

1. Baseline: generate a baseline dataset $X$ of size $N = 1000$ with $d=3$ features where each feature follows a standard normal distribution. Generate Y as a linear combination of each X. Focus on various magnitudes.  
2. Scenario 1: Covariate Shift: simulate covariate shift by altering the distribution of one or more features. E.g. change x1 from N(0,1) to N(0,2) after a certain time point T. 
3. Scenario 2: Concept Shift: simulate concept shift by changing the relationship. E.g. change the underlying function Y = \sum(X) to Y = x1^2 + x2 + x3. 
4. Scenario 3: Mixed Drift: Combine both forms by changing x1 to N(0,2) and Y to the new function.
5. Scenario 4: Linearly change the mean of X1 over time by gradually multiplying it by an increasing factor. 

**Consider PE and Traditional Methods**
1. Traditional: KS Test, Chi-Squared, Population Stability Index
2. TDA-PE

**Demonstrate Robustness**
1. Tweak hyperparameters (distance metrics, window size, etc.) 

**Compare time-to-identification by metric** 
1. Show that time to identification is better with TDA than it is with traditional metrics. 

# Simulate Dataset

In [2]:
np.random.seed(42)

df = pd.DataFrame({'x1': np.random.normal(loc=0, scale=1, size=1000),
                   'x2': np.random.normal(loc=0, scale=1, size=1000),
                   'x3': np.random.normal(loc=0, scale=1, size=1000)})
df['y'] = df['x1'] + df['x2'] + df['x3']

In [3]:
# Inject variance-drift scenarios
chg_var = {} 
chgAmts = [0.0001, 0.001, 0.01, 0.1, 0.6, 1, 5, 10, 20, 30]
for c in chgAmts: 
    tmp = df.copy(deep = True)
    new = pd.DataFrame({'x1': np.random.normal(loc=0*c, scale=1*c, size=400),
                    'x2': np.random.normal(loc=0, scale=1, size=400),
                    'x3': np.random.normal(loc=0, scale=1, size=400)})
    new['y'] = new['x1'] + new['x2'] + new['x3']
    chg_var[c] = pd.concat([tmp, new])

In [4]:
# Inject covariance shift scenarios
chg_cov_shift = {} 
chgs = ['x^2', 'x^3', 'x^4', 'x^0.5', 'x^0.25', '2x']
for c in chgs: 
    tmp = df.copy(deep = True)
    new = pd.DataFrame({'x1': np.random.normal(loc=0, scale=1, size=400),
                   'x2': np.random.normal(loc=0, scale=1, size=400),
                   'x3': np.random.normal(loc=0, scale=1, size=400)})
    if c == 'x^2': new['y'] = new['x1']**2 + new['x2'] + new['x3']
    elif c == 'x^3': new['y'] = new['x1']**3 + new['x2'] + new['x3']
    elif c == 'x^4': new['y'] = new['x1']**4 + new['x2'] + new['x3']
    elif c == 'x^0.5': new['y'] = new['x1']**0.5 + new['x2'] + new['x3']
    elif c == 'x^0.25': new['y'] = new['x1']**0.25 + new['x2'] + new['x3']
    else: new['y'] = 2*new['x1'] + new['x2'] + new['x3']
    chg_cov_shift[c] = pd.concat([tmp, new])

In [5]:
# Inject Mixed Drift
chg_mix_shift = {} 
chgs = [0.001, 0.1, 5, 30]
for c in chgAmts: 
    tmp = df.copy(deep = True)
    new = pd.DataFrame({'x1': np.random.normal(loc=0*c, scale=1*c, size=400),
                    'x2': np.random.normal(loc=0, scale=1, size=400),
                    'x3': np.random.normal(loc=0, scale=1, size=400)})
    new['y'] = new['x1']**0.5 + new['x2'] + new['x3']
    chg_mix_shift[c] = pd.concat([tmp, new])


# Apply PE

In [15]:
dicts = {'chg_var': chg_var, 'cov_shift': chg_cov_shift, 'mix_shift': chg_mix_shift}
results = {} 
for d in dicts.keys(): 
    name = d
    chg = dicts[d]
    res = {} 
    for k in chg.keys(): 
        for w in [30]: 
            print("K: " + str(k))
            print("W: " + str(w))
            # Combine df and df_drifted
            df_combined = chg[k]

            # Generate a time index
            df_combined['time'] = np.arange(len(df_combined))

            # Create a function to compute persistence diagrams
            def compute_persistence_diagrams(data):
                rips_complex = RipsComplex(points=data)
                simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)
                simplex_tree.persistence()
                persistence_intervals = simplex_tree.persistence_intervals_in_dimension(1)
                return persistence_intervals

            # Create a function to compute persistence entropy
            def compute_persistence_entropy(diagrams):
                entropies = []
                for diagram in diagrams:
                    lifetimes = diagram[:, 1] - diagram[:, 0]
                    lifetimes = lifetimes[lifetimes > 0]
                    if len(lifetimes) > 0:
                        p = lifetimes / lifetimes.sum()
                        entropy = -np.sum(p * np.log(p))
                    else:
                        entropy = 0
                    entropies.append(entropy)
                return entropies

            # Compute persistence entropy over time
            entropy_values = []

            for start in range(len(df_combined) - w + 1):
                window_data = df_combined[['x1', 'x2', 'x3', 'y']].iloc[start:start + w].values
                persistence_diagrams = [compute_persistence_diagrams(window_data)]
                entropy_value = compute_persistence_entropy(persistence_diagrams)[0]
                entropy_values.append(entropy_value)
            df_combined['pe'] = [0]*(w-1) + entropy_values
            dfc = df_combined.copy(deep = True)
            res[str(k) + '-' + str(w)] = dfc
    results[name] = res

K: 0.0001
W: 30
K: 0.001
W: 30
K: 0.01
W: 30
K: 0.1
W: 30
K: 0.6
W: 30
K: 1
W: 30
K: 5
W: 30
K: 10
W: 30
K: 20
W: 30
K: 30
W: 30
K: x^2
W: 30
K: x^3
W: 30
K: x^4
W: 30
K: x^0.5
W: 30
K: x^0.25
W: 30
K: 2x
W: 30
K: 0.0001
W: 30
K: 0.001
W: 30
K: 0.01
W: 30
K: 0.1
W: 30
K: 0.6
W: 30
K: 1
W: 30
K: 5
W: 30
K: 10
W: 30
K: 20
W: 30
K: 30
W: 30


In [16]:
with open('results_2.pkl', 'wb') as f: 
    pickle.dump(results, f)

In [6]:
from gtda.diagrams import PairwiseDistance
# WASSERSTEIN

dicts = {'chg_var': chg_var, 'cov_shift': chg_cov_shift, 'mix_shift': chg_mix_shift}
results = {} 
for d in dicts.keys(): 
    name = d
    chg = dicts[d]
    res = {} 
    for k in chg.keys(): 
        for w in [30]: 
            print("K: " + str(k))
            print("W: " + str(w))
            # Combine df and df_drifted
            df_combined = chg[k]

            # Generate a time index
            df_combined['time'] = np.arange(len(df_combined))

            # Create a function to compute persistence diagrams
            def compute_persistence_diagrams(data):
                rips_complex = RipsComplex(points=data)
                simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)
                simplex_tree.persistence()
                persistence_intervals = simplex_tree.persistence_intervals_in_dimension(1)
                return persistence_intervals

        def compute_wasserstein_distances(diagrams):
            wasserstein_distances = []
            if len(diagrams) < 2: return wasserstein_distances  # Return an empty list if there aren't enough diagrams to compare
            
            pairwise_distance = PairwiseDistance(metric='wasserstein')
            for i in range(1, len(diagrams)):
                diagram_1 = diagrams[i - 1]
                diagram_2 = diagrams[i]
        
                # Compute the Wasserstein distance between diagram_1 and diagram_2
                distance = pairwise_distance.fit_transform([diagram_1, diagram_2])[0, 1]
                wasserstein_distances.append(distance)
            return wasserstein_distances

        # Compute WD over time
        wasserstein_values = []

        for start in range(len(df_combined) - w + 1):
            window_data = df_combined[['x1', 'x2', 'x3', 'y']].iloc[start:start + w].values
            persistence_diagrams = [compute_persistence_diagrams(window_data)]
            wds = compute_wasserstein_distances(persistence_diagrams)
            wasserstein_values.append(wds)
        df_combined['wd'] = [0]*(w-1) + wasserstein_values
        dfc = df_combined.copy(deep = True)
        res[str(k) + '-' + str(w)] = dfc
    results[name] = res

K: 0.0001
W: 30
K: 0.001
W: 30
K: 0.01
W: 30
K: 0.1
W: 30
K: 0.6
W: 30
K: 1
W: 30
K: 5
W: 30
K: 10
W: 30
K: 20
W: 30
K: 30
W: 30
K: x^2
W: 30
K: x^3
W: 30
K: x^4
W: 30
K: x^0.5
W: 30
K: x^0.25
W: 30
K: 2x
W: 30
K: 0.0001
W: 30
K: 0.001
W: 30
K: 0.01
W: 30
K: 0.1
W: 30
K: 0.6
W: 30
K: 1
W: 30
K: 5
W: 30
K: 10
W: 30
K: 20
W: 30
K: 30
W: 30


In [7]:
with open('results_wd.pkl', 'wb') as f: 
    pickle.dump(results, f)